In [1]:
import os
import pandas as pd
import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import re
from tqdm import tqdm
import pdb
import pickle

url_prefix = 'http://vip.stock.finance.sina.com.cn'
url_pattern = r'href=".+target='
title_pattern = r'_blank">.+</a>'
year_pattern = r'20[01][0-9]'

code = '000001'

def getTitle(code):
    try:
        reports ={}
        company_url = f'http://vip.stock.finance.sina.com.cn/corp/go.php/vCB_Bulletin/stockid/{code}/page_type/ndbg.phtml'
        print(f'Processing {code} ')

        page_client = urlopen(company_url)
        page = page_client.read().decode('gbk')
        page_client.close()

        page_soup = soup(page, "html.parser")
        content = page_soup.find('div',{'class':'datelist'})

        all_url_title = content.findAll('a')

        current_id_set = set()
        for v in all_url_title:
            url = url_prefix + re.findall(url_pattern,str(v))[0][6:-9]
            url = url.replace('amp;','')          

            title = re.findall(title_pattern,str(v))[0][8:-4]
            year = re.findall(year_pattern, title)
            if not year: continue
            year = year[0]
            report_id = code+'_'+year
            if report_id not in reports:
                reports[report_id]=url;
                
#         print(reports)
        return reports
    except Exception as e:
        print(e)

getTitle(code)  

def readContent(url):
    try:
        page_client = urlopen(url)
        page = page_client.read().decode('gbk')
        page_client.close()

        report_soup = soup(page, "html.parser", )

        content = str(report_soup.find('pre'))
#         print(content)
        return content
    except Exception as e:
        print(e)

# url = 'http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllBulletinDetail.php?stockid=000004&id=4386458'; 
# readContent(url)

Processing 000001 
HTTP Error 456: 


In [2]:
import os
import pandas as pd
import re
from tqdm import tqdm
df1 = pd.read_excel('out_join2_20190318.xlsx',index_col=0,
            dtype={'Stkcd': str})
df1.head()



,Stkcd,Reptdt,Stknme,Listexg,Conme,Nindcd,Nindnme,Regadd,Area,Province,...,Explanation,IndustryCode,IndustryName,TotalAssets,TotalLiability,IntangibleAsset,NetProfit,OperatingEvenue,OperatingCost,OperationProfit
2,000004,2003-12-31,北大高科,2,深圳市北大高科技股份有限公司,G54,道路运输业,深圳市蛇口新时代广场27楼A-D,华南,广东省,...,NaN,G54,道路运输业,2.378752e+08,1.094558e+08,1482563.54,7641381.76,1.195350e+08,68211544.36,4712436.83
3,000004,2004-12-31,北大高科,2,深圳市北大高科技股份有限公司,G54,道路运输业,深圳市蛇口新时代广场27楼A-D,华南,广东省,...,NaN,G54,道路运输业,2.142702e+08,9.164253e+07,3146817.72,-3282932.11,1.067478e+08,69169581.00,-7519874.60
4,000004,2005-12-31,国农科技,2,深圳中国农大科技股份有限公司,G54,道路运输业,深圳市蛇口新时代广场27楼A-D,华南,广东省,...,NaN,G54,道路运输业,1.558688e+08,5.593254e+07,2862825.77,-18577413.51,6.137672e+07,37042271.75,-21888181.45
5,000004,2006-12-31,*ST 国农,2,深圳中国农大科技股份有限公司,G54,道路运输业,深圳市蛇口新时代广场27楼A-D,华南,广东省,...,NaN,G54,道路运输业,2.043046e+08,9.202187e+07,12234831.82,3055159.25,4.563900e+07,23909820.63,5426256.12
6,000004,2007-12-31,ST 国农,2,深圳中国农大科技股份有限公司,G54,道路运输业,深圳市蛇口新时代广场27楼A-D,华南,广东省,...,NaN,G54,道路运输业,1.698226e+08,7.203596e+07,12181733.60,-10001970.54,4.044954e+07,24730508.86,-9619521.35


In [3]:

col=df1.iloc[:,0]
#取表中的第3列的所有值
arrs=col.values
#输出结果
ids = list(set(arrs))
import os
import time
counter=0
for i in tqdm(ids):
    reports = getTitle(i)
    retry = 0
    while not reports and retry<3:
        print(f"获取年报地址失败:{i}, retrying..")
        time.sleep(1+1*retry*retry)
        reports = getTitle(i)
        retry += 1

    if not reports:
        print(f"获取年报地址失败:{i}",'*'*50)
        with open('error.txt','a') as f2:
            f2.write(f'{i}\n')
            f2.close()
        continue
    for (f,url) in reports.items():
        file = f"scraping/content/{f}.txt"
        if os.path.exists(file):
            continue
        content = readContent(url)
        retry = 0
        while not content and retry<3:
            print(f"获取年报地址失败:{i}, retrying..")
            time.sleep(0.1+0.5*retry*retry)
            content = readContent(url)
            retry += 1
        if not content:
            print( f"下载文件失败:{url}")
            continue
        
        print(f"下载文件成功:{url}")
        with open(file,'w') as f:
            counter += 1
            f.write(content)
        
        if counter %100 ==0:
            print('*'*80)
            print('下载文件数',counter)
            print('*'*80)

        

        

  0%|          | 0/37 [00:00<?, ?it/s]

Processing 300254
 
Remote end closed connection without response
获取年报地址失败:300254
, retrying..
Processing 300254
 
Remote end closed connection without response
获取年报地址失败:300254
, retrying..
Processing 300254
 
Remote end closed connection without response
获取年报地址失败:300254
, retrying..


  3%|▎         | 1/37 [00:08<04:55,  8.22s/it]

Processing 300254
 
Remote end closed connection without response
获取年报地址失败:300254
 **************************************************
Processing 900953
 
Remote end closed connection without response
获取年报地址失败:900953
, retrying..
Processing 900953
 
Remote end closed connection without response
获取年报地址失败:900953
, retrying..
Processing 900953
 
Remote end closed connection without response
获取年报地址失败:900953
, retrying..


  5%|▌         | 2/37 [00:16<04:47,  8.22s/it]

...

Processing 002406
 
Remote end closed connection without response
获取年报地址失败:002406
 **************************************************
Processing 000668
 
Remote end closed connection without response
获取年报地址失败:000668
, retrying..
Processing 000668
 
Remote end closed connection without response
获取年报地址失败:000668
, retrying..
Processing 000668
 
Remote end closed connection without response
获取年报地址失败:000668
, retrying..


 97%|█████████▋| 36/37 [04:56<00:08,  8.25s/it]

Processing 000668
 
Remote end closed connection without response
获取年报地址失败:000668
 **************************************************
Processing 300427
 
Remote end closed connection without response
获取年报地址失败:300427
, retrying..
Processing 300427
 
Remote end closed connection without response
获取年报地址失败:300427
, retrying..
Processing 300427
 
Remote end closed connection without response
获取年报地址失败:300427
, retrying..


100%|██████████| 37/37 [05:05<00:00,  8.25s/it]

Processing 300427
 
Remote end closed connection without response
获取年报地址失败:300427
 **************************************************


In [14]:
df2 = df1[['Stkcd','Reptdt','Stknme']] 
df2
data = df2.to_dict('records')
print(data[0])

{'Stkcd': '000004', 'Reptdt': '2003-12-31', 'Stknme': '北大高科'}
